In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import tiktoken
import json
import datetime as dt

In [3]:
load_dotenv()

api_key = os.getenv("oapi_key")
base_url = os.getenv("open_base_url")
model = os.getenv("model")

In [11]:
class Bot:
    def __init__(self,api_key, base_url = base_url,
                model = model,default_temperature = 0.7,
                default_max_tokens = 500,
                token_budget = 4096,
                 history_file = None
                ):
        self.client = OpenAI(api_key=api_key)
        self.base_url = base_url
        self.model = model
        self.token_budget = token_budget
        self.default_temperature = default_temperature
        self.default_max_tokens = default_max_tokens
        if not history_file:
            timestamp = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
            self.history_file = f"conversation_history_{timestamp}.json"
        else:
            self.history_file = history_file
        self.personas = {
                "blogger": "You are a creative blogger specializing in engaging and informative content for DigiTech Synergy Pvt. Ltd.",
                "social_media_expert": "You are a social media expert, crafting catchy and shareable posts for DigiTech Synergy Pvt. Ltd.",
                "creative_assistant": "You are a creative assistant skilled in crafting engaging marketing content for DigiTech Synergy Pvt. Ltd"
            }

        self.system_message = self.personas["creative_assistant"] # default system message
#         self.conversation_history = [{"role":"system",
#                                      "content":self.system_message}]
        self.load_conversation_history()
    
    def load_conversation_history(self):
        try:
            with open(self.history_file) as f:
                self.conversation_history = json.load(f)
        except FileNotFoundError:
            self.conversation_history = [{"role":"system",
                                          "content":self.system_message}]
            
        except json.JSONDecodeError:
            self.conversation_history = [{"role":"system","content":self.system_message}]
     
    def save_conversation_history(self):
        try:
            with open(self.history_file, "w") as f:
                json.dump(self.conversation_history,f, indent = 4)
        except IOError as e:
            print(f"An I/O error occurred while saving the conversation history: {e}")
        
        except Exception as e:
            print(f"An unexpected error occurred while saving the conversation history: {e}")
            
    def reset_conversation_history(self):
        self.conversation_history = [{"role":"system","content":self.system_message}]
        self.save_conversation_history()
    
        
    def count_tokens(self, text):
        try:
            encoding = tiktoken.encoding_for_model(self.model)
        except KeyError:
            encoding = tiktoken.get_encoding("cl100k_base")
        
        tokens = encoding.encode(text)
        return len(tokens)
    
    def total_token_used(self):

        return sum(self.count_tokens(message["content"]) for message in self.conversation_history)
    
    def enforce_token_budget(self):
        while self.total_token_used() > self.token_budget:
            if len(self.conversation_history) <= 1:
                break
            else:
                self.conversation_history.pop(1)
    
    def set_persona(self, persona):
        if persona in self.personas:
            self.system_message = self.personas[persona]
            if self.conversation_history and self.conversation_history[0]["role"] == "system":
                self.conversation_history[0]["content"] = self.system_message
            else:
                self.conversation_history.insert(0,{"role":"system",
                                                 "content":self.system_message})
            
        else:
            raise KeyError(f"""Unknwon Persona {persona}. Available \
            personas are: {list(self.personas.keys())}""")
            
    def set_custom_system_message(self, custom_message):
        if not custom_message:
            raise ValueError("Custom message can't be empty")
        else:
            self.personas["custom"] = custom_message
            
        self.set_persona("custom")
                      
    
    def chat_completion(self, prompt, temperature = None, max_tokens = None):
        
        temperature = temperature if temperature is not None else self.default_temperature
        max_tokens = max_tokens if max_tokens is not None else self.default_max_tokens
        
        self.conversation_history.append({"role":"user","content":prompt})
        
        self.enforce_token_budget()
        
        try:
        
            response = self.client.chat.completions.create(model = self.model,
                                                          messages = self.conversation_history,
                                                          temperature = temperature,
                                                          max_tokens = max_tokens)
        except Exception as e:
            print(f"An error occured while generating a response: {e}")
            return None
            

        ai_response = response.choices[0].message.content
        
        self.conversation_history.append({"role":"assistant","content":ai_response})
        self.save_conversation_history()
        return ai_response
        

In [12]:
bot = Bot(api_key, token_budget= 100)
resp = bot.chat_completion("Write a 3 lines AI marketing content")
# resp

In [13]:
bot.history_file

'conversation_history_20240226_174300.json'

In [14]:
bot.conversation_history

[{'role': 'system',
  'content': 'You are a creative assistant skilled in crafting engaging marketing content for DigiTech Synergy Pvt. Ltd'},
 {'role': 'user', 'content': 'Write a 3 lines AI marketing content'},
 {'role': 'assistant',
  'content': '"Unlock the power of AI with DigiTech Synergy Pvt. Ltd. Revolutionize your business with cutting-edge technology solutions tailored to your needs. Embrace the future of innovation today."'}]

In [15]:
bot.reset_conversation_history()

In [16]:
bot.conversation_history

[{'role': 'system',
  'content': 'You are a creative assistant skilled in crafting engaging marketing content for DigiTech Synergy Pvt. Ltd'}]